In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.patches import Circle


import timeit
import numba
from numba import jit, njit

# from iteration_methods import*
from basic import*

In [2]:
#Set output options
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
@njit
def source(r, th, u, params = (1, 0, 0.5)):

    sigma, s, uc = params

    f1 = -sigma*s**2*(u-uc)**(2*sigma-1)
    
    return f1

@njit
def u_analytical(r, th):

    uan = np.sin(th)**2/r 

    return uan

In [4]:
def gs_comp_solver(matrix, source_term, grid, init_guess=None, 
                    boundary=((0, 0), (0,0)), tolerance=1.e-8, 
                    itermax=1000, omega=1.5, params=(1, 0, 0.5)):

    #read the input
    M, u0, B, tol, kmax = matrix, init_guess, boundary, tolerance, itermax

    #assign the grid
    q, th = grid

    Q, TH = q[0], th[:,0]

    Nq, Nth = q.shape[1]-1, th.shape[0]-1

    dq, dth = (q[0, -1] - q[0, 0])/Nq, (th[-1, 0] - th[0, 0])/Nth
 
    
    
    #assign the initial guess
    if init_guess is None:

        u0 = np.ones_like(r)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                          #q_I boundary              
    u0[:, -1] = B[0][1]                         #q_F boundary
    u0[0, :] = B[1][0]                          #th_I boundary
    u0[-1, :] = B[1][1]                         #th_F boundary
    
    #assign extra parameters
    sigma, s, uc = params

    R = Q[-1]           #The radius of the star

    #assign the source term
    f = np.zeros_like(q)
    for j in range(1, Nth):
            for i in range(1, Nq):
                
                #simulate the effect of the heaviside step function
                if u0[j,i] >= uc:

                    f[j,i] = source_term(q[i], th[j], u0[j,i], params)
                else:
                    
                    f[j,i] = 0.


    #initial values before the iteration loop starts
    u = u0.copy()
    k = 0
    rel_diff = tol + 1
    conv_hist = []

    

    #iteration loop 
    while  k < kmax and rel_diff > tol:    
        
        # print the iteration number to keep track of the solver
        # if np.mod(k, 200) == 0:

        #     print(k)

        u_next = u.copy()
      
        #calculate the solution in the kth step
        for j in range(1, Nth):
            for i in range(1, Nq):
                
                #Update the source term if it is a function of the solution
                #simulate the effect of the heaviside step function
                if u[j,i] >= uc:

                    f[j,i] = source_term(q[i], th[j], u[j,i], params)
                else:
                    
                    f[j,i] = 0.


                #update the solution using SOR method
                u_next[j,i] = (1-omega)*u[j,i] + omega/(2*(Q[i]**2*dth**2 + dq**2))*  \
                              (dth**2*Q[i]*(u[j,i+1]*(Q[i]+dq) + u_next[j,i-1]*(Q[i]-dq)) 
                               +dq**2*(u[j+1,i]*(1-dth/(2*np.tan(TH[j]))) + u_next[j-1,i]*(1+dth/(2*np.tan(TH[j])))) 
                               -(dq*dth*R**2/Q[i])**2*f[j,i])
                
        #calculate the L2 norm of the relative difference between the two last iterations   
        rel_diff = la.norm(u_next-u)/la.norm(u)
        
        #Save the convergence history
        conv_hist.append(rel_diff)

        #update solution for next iteration
        u = u_next

        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
#Setup the parameters for the solver. Function grid() is in the basic.py module

#The grid of the problem
r_s = 1.
Q, TH, q, th, dq, dth = polar_grid(th_I=0, th_F=np.pi, r_I=0, r_F=r_s, Nr=150, Nth=150)            

R, r = r_s**2/Q, r_s**2/q
#initial guess
u_guess = np.sin(th)**2/r_s**2*q       

#extra parameters that may be needed
sigma = 1.1
s = 0.
uc = 0.5
params = (sigma , s, uc)
params1 = (sigma , 0., uc)
params2 = (1. , s, uc)

#source term
# f = source_1(r, th, u_guess, params)      

#analytical solution for comparison
u_an = u_analytical(r, th)

#boundary conditions
boundary = ((0, np.sin(TH)**2/r_s), (0., 0.))                                   

#max number of iteration
iterations = 10000                              

#desired tolerance
tolerance = 1.e-8                               

#relaxation parameter for SOR method
omega_opt = 2/(1+np.sin(np.pi*max(dq, dth)))     

In [6]:
#jit the solver function
gs_comp_solver = jit(nopython=True)(gs_comp_solver)

In [7]:
#Calculate, time and qualify the solution

start = timeit.default_timer()

u, k, rel_diff, conv_hist = gs_comp_solver(np.eye(2), source, (q, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

# u1, k1, rel_diff1, conv_hist1 = gs_comp_solver(np.eye(2), source, (q, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params1)

# u2, k2, rel_diff2, conv_hist2 = gs_comp_solver(np.eye(2), source, (q, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params2)

elapsed = timeit.default_timer() - start

#Relative error to analytical
error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

#print solution
print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

SOR solver 
Number of iterations: 458 
Last Relative difference: 9.960E-09
Error to analytical: 6.638E-05s 
Elapsed time: 6.42E+00s 
--------------------------------------------------


In [8]:
#Plot results. Function plot() is in basic.py module.
# from basic import plot

# plot(r*np.cos(th), r*np.sin(th), u, u1=None, u_an=None, conv_hist=conv_hist, plot_result=False)

In [9]:
fig2, ax2 = plt.subplots()

levels=20
colormap=cm.viridis

cont = ax2.contour(r*np.sin(th), r*np.cos(th), u, levels=levels, cmap=colormap)

# cont1 = ax2.contour(r*np.sin(th), r*np.cos(th), u1, levels=levels, cmap=colormap, linestyles = 'dashed')

# cont2 = ax2.contour(r*np.sin(th), r*np.cos(th), u2, levels=levels, cmap=colormap, linestyles = 'dotted') 

# cont_c = ax2.contour(r_s**2/q*np.sin(th), r_s**2/q*np.cos(th), u, levels=[uc], colors='red')
# cont_c1 = ax2.contour(r*np.sin(th), r*np.cos(th), u1, levels=[uc], colors='green')
# cont_c2 = ax2.contour(r*np.sin(th), r*np.cos(th), u2, levels=[uc], colors='yellow')
cont_an = ax2.contour(r*np.sin(th), r*np.cos(th), u_an, levels=levels, cmap=colormap, alpha=0.5)

ax2.add_patch(Circle((0.,0.), r_s, color='b', zorder=100))

ax2.set_ylim(top=10*r_s*np.cos(TH[0])/2, bottom=10*r_s*np.cos(TH[-1])/2)
ax2.set_xlim(left=0., right=10*r_s)

ax2.set_title('compact')
# h,_ = cont.legend_elements()
# # h1,_ = cont1.legend_elements()
# # h2,_ = cont2.legend_elements()
# h_an,_ = cont_an.legend_elements()
# ax2.legend([h[0], h2[0], h1[0], h_an[0]], ['twisted', 'linear', 'vacuum', 'analytical'], loc=3)

Text(0.5, 1.0, 'compact')